In [3]:
import matplotlib.pyplot as plt
import autograd.numpy as np

from matplotlib.colors import LogNorm
from matplotlib import animation
from IPython.display import HTML

In [7]:
from autograd import elementwise_grad, value_and_grad
from scipy.optimize import minimize
from collections import defaultdict
#from itertools import izip_longest
from functools import partial

from mpl_toolkits.mplot3d import axes3d, Axes3D
from matplotlib import cm

import numpy as np

In [9]:
try:
    # Python 3
    from itertools import zip_longest
except ImportError:
    # Python 2
    from itertools import izip_longest as zip_longest

In [11]:
from numdifftools import Jacobian, Hessian

def fun(x):
    return (x[0]**2 + x[1]**2) + (0.5*x[0] + x[1])**2 + (0.5*x[0] + x[1])**4

def fun_der(x):
    return Jacobian(lambda x: fun(x))(x).ravel()

def fun_hess(x):
    return Hessian(lambda x: fun(x))(x)

def make_minimize_cb(path=[]):
    def minimize_cb(xk):
        path.append(np.copy(xk))
    return minimize_cb


def f(x, y):
    return (x**2 + y**2) + (0.5*x + y)**2 + (0.5*x + y)**4

minima = np.array([0, 0])
start_point = np.array([8, 8])
minima_ = minima.reshape(-1, 1)

xmin, xmax = -10, 10
ymin, ymax = xmin, xmax

print(f(*minima))

X = np.arange(xmin, xmax, 0.25)
Y = np.arange(xmin, xmax, 0.25)
X, Y = np.meshgrid(X, Y)
Z = f(X, Y)

ModuleNotFoundError: No module named 'numdifftools'

In [3]:
methods = ['trust-ncg', 'dogleg']

minimize_ = partial(minimize, fun=fun, x0=start_point, jac=fun_der, hess=fun_hess, bounds=[(-4.5, 4.5), (-4.5, 4.5)], tol=1e-20)

paths_ = defaultdict(list)
for method in methods:
    paths_[method].append(start_point)
    
results = {method: minimize_(method=method, callback=make_minimize_cb(paths_[method])) for method in methods}

paths = [np.array(paths_[method]).T for method in methods]

zpaths = [fun(path) for path in paths]

/anaconda2/lib/python2.7/site-packages/scipy/optimize/_minimize.py:430: RuntimeWarning: Method trust-ncg cannot handle constraints nor bounds.
  RuntimeWarning)
/anaconda2/lib/python2.7/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
/anaconda2/lib/python2.7/site-packages/scipy/optimize/_minimize.py:430: RuntimeWarning: Method dogleg cannot handle constraints nor bounds.
  RuntimeWarning)


In [4]:
class TrajectoryAnimation(animation.FuncAnimation):
    
    def __init__(self, paths, labels=[], fig=None, ax=None, frames=None, 
                 interval=60, repeat_delay=5, blit=True, **kwargs):

        if fig is None:
            if ax is None:
                fig, ax = plt.subplots()
            else:
                fig = ax.get_figure()
        else:
            if ax is None:
                ax = fig.gca()

        self.fig = fig
        self.ax = ax
        
        self.paths = paths

        if frames is None:
            frames = max(path.shape[1] for path in paths)
  
        self.lines = [ax.plot([], [], label=label, lw=2)[0] 
                      for _, label in izip_longest(paths, labels)]
        self.points = [ax.plot([], [], 'o', color=line.get_color())[0] 
                       for line in self.lines]

        super(TrajectoryAnimation, self).__init__(fig, self.animate, init_func=self.init_anim,
                                                  frames=frames, interval=interval, blit=blit,
                                                  repeat_delay=repeat_delay, **kwargs)

    def init_anim(self):
        for line, point in zip(self.lines, self.points):
            line.set_data([], [])
            point.set_data([], [])
        return self.lines + self.points

    def animate(self, i):
        for line, point, path in zip(self.lines, self.points, self.paths):
            line.set_data(*path[::,:i])
            point.set_data(*path[::,i-1:i])
        return self.lines + self.points

In [5]:
class TrajectoryAnimation3D(animation.FuncAnimation):
    
    def __init__(self, paths, zpaths, labels=[], fig=None, ax=None, frames=None, 
                 interval=60, repeat_delay=5, blit=True, **kwargs):

        if fig is None:
            if ax is None:
                fig, ax = plt.subplots()
            else:
                fig = ax.get_figure()
        else:
            if ax is None:
                ax = fig.gca()

        self.fig = fig
        self.ax = ax
        
        self.paths = paths
        self.zpaths = zpaths
        
        if frames is None:
            frames = max(path.shape[1] for path in paths)
  
        self.lines = [ax.plot([], [], [], label=label, lw=2)[0] 
                      for _, label in izip_longest(paths, labels)]

        super(TrajectoryAnimation3D, self).__init__(fig, self.animate, init_func=self.init_anim,
                                                  frames=frames, interval=interval, blit=blit,
                                                  repeat_delay=repeat_delay, **kwargs)

    def init_anim(self):
        for line in self.lines:
            line.set_data([], [])
            line.set_3d_properties([])
        return self.lines

    def animate(self, i):
        for line, path, zpath in zip(self.lines, self.paths, self.zpaths):
            line.set_data(*path[::,:i])
            line.set_3d_properties(zpath[:i])
        return self.lines

In [6]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.contour(X, Y, Z, levels=np.logspace(0, 5, 35), norm=LogNorm(), cmap=cm.gray)
ax.plot([minima[0]], [minima[1]], 'r*', markersize=10)

ax.set_xlabel('$x$')
ax.set_ylabel('$y$')

ax.set_xlim((xmin, xmax))
ax.set_ylim((ymin, ymax))

anim = TrajectoryAnimation(paths, labels=methods, ax=ax)

ax.legend(loc='upper left')


# HTML(anim.to_html5_video())

anim.save('./gifs3/zakharov2.gif', dpi=80, writer='imagemagick')

/anaconda2/lib/python2.7/site-packages/matplotlib/contour.py:1537: UserWarning: Log scale: values of z <= 0 have been masked
  warnings.warn('Log scale: values of z <= 0 have been masked')


In [8]:
fig = plt.figure(figsize=(8, 5))
ax = plt.axes(projection='3d', elev=50, azim=-50)

ax.plot_surface(X, Y, Z, norm=LogNorm(), rstride=1, cstride=1, edgecolor='none', alpha=.8, cmap=cm.gray)
ax.plot([minima[0]], [minima[1]], 'r*', markersize=10)

ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')

ax.set_xlim((xmin,xmax))
ax.set_ylim((ymin, ymax))

anim = TrajectoryAnimation3D(paths, zpaths=zpaths, labels=methods, ax=ax)

ax.legend(loc='upper left')


# HTML(anim.to_html5_video())

anim.save('./gifs3/zakharov1.gif', dpi=80, writer='imagemagick')